In [1]:
import os
import math
from os import path

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import TensorBoard, ReduceLROnPlateau
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.applications.vgg19 import VGG19
from sklearn.model_selection import train_test_split
from PIL import Image

# A DEFINIR AVANT L'ENTRAINEMENT ET L'INFERENCE
batch_size = 8

In [2]:
tf.test.is_gpu_available()

True

# GITW Transfer Learning 

# Data Loading

In [2]:
gitw_path = path.join("data", "GITW_clean")
classes = [d for d in os.listdir(gitw_path) if path.isdir(path.join(gitw_path, d))]
classes

['Background',
 'Bowl',
 'CanOfCocaCola',
 'FryingPan',
 'Glass',
 'Jam',
 'Lid',
 'MilkBottle',
 'Mug',
 'OilBottle',
 'Plate',
 'Rice',
 'SaucePan',
 'Sponge',
 'Sugar',
 'VinegarBottle',
 'WashLiquid']

In [3]:
directories = []
files = {
    "file": [],
    "label": [],
    "frame": []
}

for c_name in classes:
    for img in os.listdir(path.join(gitw_path, c_name)):
        if path.isfile(path.join(gitw_path, c_name, img)):
            files["file"].append(path.join(c_name, img))
            files["label"].append(classes.index(c_name))
            files["frame"].append(int(img.split("_")[1]))

In [4]:
files = pd.DataFrame(files).sort_values(by=["label", "frame"]).reset_index(drop=True)
files.head()

,file,label,frame
0,Background\Frame_0320_bg.png,0,320
1,Background\Frame_0360_bg.png,0,360
2,Background\Frame_0400_bg.png,0,400
3,Background\Frame_0440_bg.png,0,440
4,Background\Frame_0480_bg.png,0,480


In [5]:
x_train, x_test = train_test_split(files.index, test_size=0.15)
len(x_train), len(x_test)

(4451, 786)

Les liens vers les donnnées de test et d'entraînement sont sauvegardés dans `train_dataset.csv` et `test_dataset.csv`

In [6]:
train_df, test_df = files.loc[x_train], files.loc[x_test]
train_df.to_csv(path.join(gitw_path, "train_dataset.csv"), index=False)
test_df.to_csv(path.join(gitw_path, "test_dataset.csv"), index=False)

Les classes GITWDataset et GITWInferenceSet permettent de charger les données depuis le dossier racine dynamiquement, pour l'entraînement comme pour l'inférence. Les deux objets génèrent des *batchs* de taille `batch_size` contenant des tuples (image, classe) pour l'entraînement et des images pour l'inférence. 

In [7]:
class GITWDataset(keras.utils.Sequence):
    """
    Generateur de données d'entraînement
    """
    def __init__(self, data_path, df, classes, batch_size=32):
        self.X = [path.join(data_path, file) for file in df["file"].values]
        self.Y = keras.utils.to_categorical(df["label"].values)
        self.batch_size = batch_size
        self.path = data_path
        self.df = df
        self.classes = classes
        
    def __len__(self):
        return math.ceil(len(self.df) / self.batch_size)
    
    def __getitem__(self, idx):
        files_x = self.X[idx*self.batch_size:(idx+1)*self.batch_size]
        batch_y = self.Y[idx*self.batch_size:(idx+1)*self.batch_size]
        batch_x = []
        for file in files_x:
            frame = np.array(Image.open(os.path.join(file)).convert("RGB").resize((224,224), Image.BICUBIC).getdata())
            batch_x.append(frame.reshape(224, 224, 3) / 255) # normalisation
        return np.stack(batch_x), batch_y
    
class GITWInferenceSet(keras.utils.Sequence):
    """
    Générateur de données pour l'inférence
    """
    def __init__(self, data_path, df, classes, batch_size=32):
        self.X = [path.join(data_path, file) for file in df["file"].values]
        self.Y = keras.utils.to_categorical(df["label"].values)
        self.batch_size = batch_size
        self.path = data_path
        self.df = df
        self.classes = classes
        
    def __len__(self):
        return math.ceil(len(self.df) / self.batch_size)
    
    def __getitem__(self, idx):
        files_x = self.X[idx*self.batch_size:(idx+1)*self.batch_size]
        batch_x = []
        for file in files_x:
            frame = np.array(Image.open(os.path.join(file)).convert("RGB").resize((224,224), Image.BICUBIC).getdata())
            batch_x.append(frame.reshape(224, 224, 3) / 255)
        return np.stack(batch_x)

In [8]:
trainset = GITWDataset(gitw_path, train_df, classes, batch_size=batch_size)
testset = GITWDataset(gitw_path, test_df, classes, batch_size=batch_size)
inferenceset = GITWInferenceSet(gitw_path, test_df, classes, batch_size=batch_size)

# ResNet

In [10]:
resnet_module = ResNet50(include_top=False, input_shape=(224, 224, 3), )
resnet = models.Sequential([
    resnet_module,
    layers.Flatten(),
    layers.Dense(len(classes), activation="softmax")
])

In [11]:
resnet.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten (Flatten)            (None, 100352)            0         
_________________________________________________________________
dense (Dense)                (None, 17)                1706001   
Total params: 25,293,713
Trainable params: 25,240,593
Non-trainable params: 53,120
_________________________________________________________________


In [12]:
tensorboard = TensorBoard(log_dir=path.join(".", "logs", "resnet"), update_freq=50) # Logs d'entraînement
reducer = ReduceLROnPlateau(monitor='val_accuracy') # Si l'accuracy de validation stagne, réduit le lr d'un facteur 0.1

optimizer = optimizers.Adam(lr=1e-3, decay=1e-8) # choisi pour son efficacité et parce qu'il a déjà servi à entraîner ResNet et VGG

resnet.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
resnet_history = resnet.fit_generator(trainset, 
                    epochs=20,
                    validation_data=testset, # La validation est effectuée avec les données de test directement
                    callbacks=[tensorboard, reducer])

Epoch 1/20
279/279 [==============================] - 297s 1s/step - loss: 3.6505 - accuracy: 0.3876 - val_loss: 13.3320 - val_accuracy: 0.0980
Epoch 2/20
279/279 [==============================] - 292s 1s/step - loss: 2.2665 - accuracy: 0.6129 - val_loss: 2.4957 - val_accuracy: 0.5407
Epoch 3/20
279/279 [==============================] - 300s 1s/step - loss: 2.0806 - accuracy: 0.6457 - val_loss: 14.3799 - val_accuracy: 0.0980
Epoch 4/20
279/279 [==============================] - 289s 1s/step - loss: 2.1587 - accuracy: 0.6284 - val_loss: 1.5320 - val_accuracy: 0.6056
Epoch 5/20
279/279 [==============================] - 290s 1s/step - loss: 2.3707 - accuracy: 0.5904 - val_loss: 1.1454 - val_accuracy: 0.6858
Epoch 6/20
279/279 [==============================] - 293s 1s/step - loss: 1.8323 - accuracy: 0.7261 - val_loss: 3.4437 - val_accuracy: 0.6272
Epoch 7/20
279/279 [==============================] - 288s 1s/step - loss: 1.5407 - accuracy: 0.7965 - val_loss: 1.8656 - val_accuracy: 0.73

# VGG19 - Blocks 4 et 5

In [9]:
vgg_module = VGG19(include_top=False, input_shape=(224, 224, 3))

# Les paramètres des blocs 1 à 3 sont fixés
for layer in vgg_module.layers:
    if ("block1" in layer.name) or ("block2" in layer.name) or ("block3" in layer.name):
        layer.trainable = False

In [10]:
vgg = models.Sequential([
    vgg_module,
    layers.Flatten(),
    layers.Dense(len(classes), activation="softmax")
])

vgg.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 7, 7, 512)         20024384  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 17)                426513    
Total params: 20,450,897
Trainable params: 18,125,329
Non-trainable params: 2,325,568
_________________________________________________________________


In [11]:
tensorboard = TensorBoard(log_dir=path.join(".", "logs", "vgg"), update_freq=50)
reducer = ReduceLROnPlateau(monitor='val_accuracy')

optimizer = optimizers.Adam(lr=1e-4, decay=1e-8)

vgg.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
vgg_history = vgg.fit_generator(trainset, 
                    epochs=20,
                    validation_data=testset,
                    callbacks=[tensorboard, reducer])

Epoch 1/20
557/557 [==============================] - 537s 964ms/step - loss: 2.6952 - accuracy: 0.1968 - val_loss: 2.1852 - val_accuracy: 0.2888
Epoch 2/20
557/557 [==============================] - 547s 982ms/step - loss: 1.3882 - accuracy: 0.5918 - val_loss: 0.9313 - val_accuracy: 0.7226
Epoch 3/20
557/557 [==============================] - 534s 959ms/step - loss: 0.7203 - accuracy: 0.7998 - val_loss: 0.6092 - val_accuracy: 0.8168
Epoch 4/20
557/557 [==============================] - 539s 967ms/step - loss: 0.4757 - accuracy: 0.8598 - val_loss: 0.5070 - val_accuracy: 0.8639
Epoch 5/20
557/557 [==============================] - 528s 948ms/step - loss: 0.3957 - accuracy: 0.8913 - val_loss: 0.5349 - val_accuracy: 0.8588
Epoch 6/20
557/557 [==============================] - 535s 961ms/step - loss: 0.2947 - accuracy: 0.9144 - val_loss: 0.4917 - val_accuracy: 0.8664
Epoch 7/20
557/557 [==============================] - 533s 957ms/step - loss: 0.2401 - accuracy: 0.9299 - val_loss: 0.5290 -

# VGG19 - Block 5

In [9]:
vgg_module = VGG19(include_top=False, input_shape=(224, 224, 3))

for layer in vgg_module.layers:
    if ("block1" in layer.name) or ("block2" in layer.name) or ("block3" in layer.name) or ("block4" in layer.name):
        layer.trainable = False

In [10]:
vgg = models.Sequential([
    vgg_module,
    layers.Flatten(),
    layers.Dense(len(classes), activation="softmax")
])

vgg.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 7, 7, 512)         20024384  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 17)                426513    
Total params: 20,450,897
Trainable params: 9,865,745
Non-trainable params: 10,585,152
_________________________________________________________________


In [11]:
tensorboard = TensorBoard(log_dir=path.join(".", "logs", "vgg_bl5"), update_freq=50)
reducer = ReduceLROnPlateau(monitor='val_accuracy')

optimizer = optimizers.Adam(lr=1e-4, decay=1e-8)

vgg.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
vgg_history = vgg.fit_generator(trainset, 
                    epochs=20,
                    validation_data=testset,
                    callbacks=[tensorboard, reducer])

Epoch 1/20
557/557 [==============================] - 545s 978ms/step - loss: 1.0510 - accuracy: 0.7001 - val_loss: 0.6144 - val_accuracy: 0.8333
Epoch 2/20
557/557 [==============================] - 539s 967ms/step - loss: 0.3816 - accuracy: 0.8865 - val_loss: 0.5018 - val_accuracy: 0.8753
Epoch 3/20
557/557 [==============================] - 562s 1s/step - loss: 0.2767 - accuracy: 0.9216 - val_loss: 0.4459 - val_accuracy: 0.8766
Epoch 4/20
557/557 [==============================] - 546s 980ms/step - loss: 0.1987 - accuracy: 0.9389 - val_loss: 0.5223 - val_accuracy: 0.8969
Epoch 5/20
557/557 [==============================] - 542s 973ms/step - loss: 0.1676 - accuracy: 0.9463 - val_loss: 0.4018 - val_accuracy: 0.9160
Epoch 6/20
557/557 [==============================] - 534s 958ms/step - loss: 0.1387 - accuracy: 0.9578 - val_loss: 0.5502 - val_accuracy: 0.8855
Epoch 7/20
557/557 [==============================] - 537s 964ms/step - loss: 0.1308 - accuracy: 0.9641 - val_loss: 0.4725 - va